In [1]:
# d for coin futures, f for usd s futures
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import requests as re
import pandas as pd
from functools import reduce

api = 'https://api.binance.com/api/v3/klines'
g_api_key='eJBYUpKfnUCA5wXDfJTKGPNaOoyLtnXIb9D6GKFbnABLCndVRRYJrdyI9rMi6D54'
g_secret_key='970sbCiGtH1W4rPW56I4MBbD9brKZegyveedljt3U3D5hry7zMbxa4sxacA80kUv'
headers = {"Accept": "application/json"}

epoch = datetime.utcfromtimestamp(0)

def dt_to_ms(dt):
    delta = dt - epoch
    return int(delta.total_seconds() * 1000)

def dt_from_ms(ms):
    return datetime.utcfromtimestamp(ms / 1000.0)

def json_to_df(json):
    df = {'time':[],'open':[],'high':[],'low':[],'close':[],'volume':[]}
    for item in json:
        df['time'].append(dt_from_ms(item[0]))
        df['open'].append(float(item[1]))
        df['high'].append(float(item[2]))
        df['low'].append(float(item[3]))
        df['close'].append(float(item[4]))
        df['volume'].append(float(item[5]))
    return pd.DataFrame(df)

def save_positions(df, columns, data):
    for i in range(len(columns)):
        df[columns[i]].append(data[i])
    return df

In [2]:
# save down data
currTime = dt_to_ms(datetime.now())
fpath = './data/'
pair = 'LTCUSDT'
interval = '1h'
start_date = '20210101'
start_year, start_month, start_day = int(start_date[0:4]), int(start_date[4:6]), int(start_date[6:8])
start_date = dt_to_ms(datetime(start_year, start_month, start_day))
limit = 1000
dfs = []
while start_date < currTime:
    url = api + '?symbol='+pair+'&interval='+interval+'&startTime='+str(start_date)+'&limit='+str(limit)
    r = re.get(url, headers=headers)
    result = r.json()
    df = json_to_df(result)
    dfs.append(df)
    start_date = dt_to_ms(pd.to_datetime(df.tail(1)['time'].values[0])) + 1*60*60*1000
df_tot = reduce(lambda x,y: x.append(y), dfs)
df_tot = df_tot.sort_values('time')
df_tot.set_index('time',inplace=True)
df_tot.to_csv(fpath+pair+'.csv')

In [68]:
# save down data uniswap
# [cumFees, cumIL, tokenPriceA, tokenPriceB, timestamp]
tokenA, tokenB = "WETH", "USDT"
interval = "1h"
lookback = "120d"
url = "http://3.131.198.9:3003/api/market/pool/backtest/uniswap/"+tokenA+"/"+tokenB+"/"+lookback+"/"+interval
r = re.get(url).json()

In [70]:
df = {'time':[],'tokenPriceA':[],'tokenPriceB':[],'close':[],'cumFees':[],'cumIL':[]}
for item in r['data']:
    df = save_positions(df, ['time','tokenPriceA','tokenPriceB','close','cumFees','cumIL'], \
                       [dt_from_ms(item[-1]*1000), item[2], item[3], item[2] / item[3], item[0], item[1]])
df_tot = pd.DataFrame(df)
df_tot = df_tot.sort_values('time')
df_tot.set_index('time',inplace=True)
df_tot.to_csv(fpath+tokenA+tokenB+'.csv')

In [71]:
import sys
import logging

class Algo:

    def init(self):
        self.logger = logging.getLogger('Algo')
        self.keys = []
        self.data = []
        return

    def ParseConfig(self, config):
        """ Parse Input Config """

        ##symbols are formated like EXCHANGE-SYMBOL (exchange specific grammar)
        self.symbol = config["symbol"]
        #self.fields = config["fields"]
        self.sdt = config["sdt"]
        self.edt = config["edt"]
        self.data_dir = config["data_dir"]
        #self.file_prefix = config['file_prefix']
        #self.data_index = config['data_index']
        #self.data_loader = DataLoader(self.symbols, self.fields)
        self.logger.info('Config Parsed')

        return

    def GetData(self):
        """ Collect Data -- According to specified symbols and fields """
        df = pd.read_csv(self.data_dir+'/'+self.symbol+'.csv')
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        self.data = df
        #self.data = self.data_loader.GetData(self.data_dir, self.file_prefix, self.data_index, self.sdt, self.edt)
        #TODO -- Implement date filter later
        # self.data = self.data[self.data['dt'] >= self.sdt]
        # self.data = self.data[self.data['dt'] <= self.edt]

        #self.keys = sorted(list(self.data.keys())[1:])
        #self.labels = []
        #for key in self.keys:
        #    self.labels.append(key.split(":")[1].split("@")[0])

        self.logger.info('Data Grab Successful')

        return

    def GenerateSignal(self):
        """ Signal Generation is Strategy Specific """

        pass

    def ConstructPortfolio(self):
        """ Position Construction is Strategy Specific """

In [72]:
from numpy.random import normal
import numpy as np
import pandas as pd
import math

def price_simulation(start_value, drift, diffusion, n_periods):
    returns = [1 + (drift + diffusion * normal()) for _ in range(n_periods)]
    prices = [start_value * np.prod(returns[:i]) for i in range(len(returns))]
    return (returns, prices)

def price_simulation_gbm(start_value, drift, diffusion, n_periods):
    prices = []
    for _ in range(n_periods):
        start_value *= math.exp((drift - (diffusion**2) / 2) + diffusion * normal())
        prices.append(start_value)
    return prices

def monte_carlo_simulation(start_value, drift, diffusion, n_periods, n_simulations):
    results = []
    for _ in range(n_simulations):
        prices = price_simulation_gbm(start_value, drift, diffusion, n_periods)
        results.append(prices)
    terminal_values = [x[-1] for x in results]
    return terminal_values

def get_bounds(values, pct):
    diff = 1. - pct
    percentiles = np.percentile(values, [100*round(diff / 2, 3), 100*round(1-(diff / 2),3)])
    return (percentiles[0], percentiles[1])

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [94]:
from datetime import datetime
import logging, sys, numpy as np, pandas as pd
import scipy.stats as st

class UniswapV3(Algo):

    def __init__(self):

        super().__init__()
        self.logger = logging.getLogger("UniswapV3")

        return

    def ParseConfig(self, config):
        """ Parse Input Config """

        super().ParseConfig(config)
        self.rolling_window = config["rolling_window"]
        self.does_stop = config["does_stop"]
        self.stop_pct = config["stop_pct"]
        self.size = config["size"]
        self.n_simulations = config["n_simulations"]
        self.n_periods = config["n_periods"]
        self.pct = config["pct"]
        self.use_drift = config["use_drift"]
        return

    def GenerateSignal(self):
        """ Generate Signal for input to Portfolio Construction """
        
        self.logger.info("Generating signal")
        data = self.data
        data['returns'] = data['close'].pct_change(1)
        data['rolling_mean'] = data['returns'].rolling(self.rolling_window).mean()
        data['rolling_std'] = data['returns'].rolling(self.rolling_window).std()
        self.data = data
        pass

        return

    def GeneratePositions(self):
        """ Generate Signal for input to Portfolio Construction """

        self.logger.info("Generating positions")
        # Downsample data according to config param
        
        positions = {'time':[],'type':[],'fracSize':[], 'low':[],'high':[], 'note':[],'tokenPriceA':[],'tokenPriceB':[], 'cumFees':[]}
        start_date = find_nearest(list(self.data.index), self.sdt)
        start_date_i = start_date
        end_date = start_date
        stop_left, stop_right = np.nan, np.nan
        left, right = np.nan, np.nan
        for i in range(self.data.shape[0]):
            date = self.data.index[i]
            if date > self.edt:
                break
            if date < start_date_i:
                continue
            close = self.data.iloc[i]['close']
            if date < end_date:
                # check if stopping out
                if ((close > stop_right) or (close < stop_left)):
                    px = self.data.iloc[i]['close']
                    tokenPriceA, tokenPriceB = self.data.iloc[i]['tokenPriceA'], self.data.iloc[i]['tokenPriceB']
                    drift = self.data.iloc[i]['rolling_mean'] if self.use_drift else 0
                    # calculate fees during interval
                    fees = self.data.iloc[i]['cumFees']
                    diffusion = self.data.iloc[i]['rolling_std']
                    terminal_values = monte_carlo_simulation(px, drift, diffusion, self.n_periods, self.n_simulations)
                    left, right = get_bounds(terminal_values, self.pct)
                    positions = save_positions(positions, ['time','type','fracSize','low','high','note','tokenPriceA','tokenPriceB', 'cumFees'], [date, 'base',  self.size, left, right, 'stop', tokenPriceA, tokenPriceB, fees])
                    end_date = dt_from_ms(dt_to_ms(date) + 24*60*60*1000)
                    start_date = date
                    if self.does_stop:
                        stop = (1-self.stop_pct) / 2
                        stop_left, stop_right = left + stop * (right-left), right - stop * (right-left)
                else:

                    tokenPriceA, tokenPriceB = self.data.iloc[i]['tokenPriceA'], self.data.iloc[i]['tokenPriceB']
                    fees = self.data.iloc[i]['cumFees']
                    positions = save_positions(positions, ['time','type','fracSize','low','high','note','tokenPriceA','tokenPriceB', 'cumFees'], [date, 'base',  self.size, left, right, 'hold', tokenPriceA, tokenPriceB, fees])

            elif date == end_date:
                # set new position
                px = self.data.iloc[i]['close']
                tokenPriceA, tokenPriceB = self.data.iloc[i]['tokenPriceA'], self.data.iloc[i]['tokenPriceB']
                drift = self.data.iloc[i]['rolling_mean'] if self.use_drift else 0
                # calculate fees during interval
                fees = self.data.iloc[i]['cumFees']
                diffusion = self.data.iloc[i]['rolling_std']
                terminal_values = monte_carlo_simulation(px, drift, diffusion, self.n_periods, self.n_simulations)
                left, right = get_bounds(terminal_values, self.pct)
                positions = save_positions(positions, ['time','type','fracSize','low','high','note','tokenPriceA','tokenPriceB','cumFees'], [date, 'base',  self.size, left, right, 'new', tokenPriceA, tokenPriceB, fees])
                end_date = dt_from_ms(dt_to_ms(date) + 24*60*60*1000)
                start_date = date
                if self.does_stop:
                    stop = (1-self.stop_pct) / 2
                    stop_left, stop_right = left + stop * (right-left), right - stop * (right-left)
            else:
                print("data issue")

        positions = pd.DataFrame(positions)
        self.positions = positions

        return


if __name__ == "__main__":

    logging.basicConfig()
    logging.getLogger().setLevel(logging.INFO)

    algo = UniswapV3()
    data_dir = "./data/"
    positions_dir = "./data/positions/"
    #exchange = "binance-futures"
    #file_prefix = "derivative_ticker"

    #coins = market_coins[key]
    symbol = 'WETHUSDT'
    algo.ParseConfig({"symbol": symbol,
                      "sdt": datetime(2021, 4, 1),  # start date
                      "edt": datetime(2021, 5, 11),  # end date
                      "size": 1,
                      "data_dir": data_dir,
                      "rolling_window": 24*30,
                      "does_stop": True,
                      "stop_pct": 0.95,
                      "n_simulations": 1000,
                      "n_periods": 24,
                      "pct": 0.95,
                      "use_drift": False
                      })

    algo.GetData()
    algo.GenerateSignal()
    algo.GeneratePositions()
    algo.positions.to_csv(positions_dir + symbol+'.csv')

    #algo.data = PnlCalculator().CalculatePnL(algo.data, algo.symbols)
    #algo.data.to_csv(positions_dir + "/MeanReversionv1_test_" + key + ".csv")


INFO:UniswapV3:Config Parsed
INFO:UniswapV3:Data Grab Successful
INFO:UniswapV3:Generating signal
INFO:UniswapV3:Generating positions


In [95]:
algo.positions

,time,type,fracSize,low,high,note,tokenPriceA,tokenPriceB,cumFees
0,2021-03-31 23:58:15.211,base,1,1780.207489,2089.991007,new,1927.015326,0.999990,0.105989
1,2021-04-01 00:58:15.211,base,1,1780.207489,2089.991007,hold,1937.479837,1.001623,0.106034
2,2021-04-01 01:58:15.211,base,1,1780.207489,2089.991007,hold,1929.651239,1.000854,0.106082
3,2021-04-01 02:58:15.211,base,1,1780.207489,2089.991007,hold,1916.531511,0.999158,0.106113
4,2021-04-01 03:58:15.211,base,1,1780.207489,2089.991007,hold,1910.549041,1.000058,0.106153
...,...,...,...,...,...,...,...,...,...
956,2021-05-10 19:58:15.211,base,1,3649.274096,4567.196359,hold,3872.975062,1.001081,0.161779
957,2021-05-10 20:58:15.211,base,1,3649.274096,4567.196359,hold,4015.854062,1.000225,0.161884
958,2021-05-10 21:58:15.211,base,1,3649.274096,4567.196359,hold,4019.676600,1.000033,0.162065
959,2021-05-10 22:58:15.211,base,1,3649.274096,4567.196359,hold,3952.992982,1.000151,0.162230
